# Web Crawler API Onibus Rio - CSV Version

# Version 01: <-
    * Busca conteúdo da API da Prefeitura do Rio de 60 em 60 segundos.
    * Salva arquivo .csv e realiza append a cada 60 segundos no arquivo .csv
    * Create HTTP and Json exceptions
    * Save file continuously without header <-
# Version 02:
    * Drop all duplicates.
    * Define time to cut data to write into file

In [7]:
import csv, time, json, pandas as pd, requests as req

In [8]:
%config IPCompleter.greedy=True

In [9]:
def window_wait():
    time.sleep(60);

In [10]:
#logerrorpath = '/data/dumps/log-errors/'
logerrorpath = './'
csvfile = './datariobus.csv'
#csvfile = '/data/dumps/datario-bus/datariobus.csv'
url = 'http://dadosabertos.rio.rj.gov.br/apiTransporte/apresentacao/rest/index.cfm/obterTodasPosicoes'

def remove_duplicates(df):
    df = pd.DataFrame
    global df_total
    df_total = pd.DataFrame
    df_total.append(df)    
    df_total.drop_duplicates(keep="first",inplace=True)
    return df_total

In [ ]:
continua=1
rodada = 0
while (continua==1):
        try:
            response = req.get(url)
            json_api = response.json()
            json_header = json_api['COLUMNS']
            json_data = json_api['DATA']
            df = pd.DataFrame(json_data, columns=json_header)
            with open(csvfile, "a+") as fcsv:
                df.set_index('DATAHORA', inplace=True)
                df.to_csv(fcsv,sep=";",header=None)
        except (Exception, req.exceptions.HTTPError) as httpError:
            http_error_time = time.strftime('%Y%m%d-%H%M%S')       
            with open(logerrorpath+'datario-http-error-msg.txt','+a') as f:
                f.write("%a\n%a\n%a\n==========\n" % (http_error_time, str(response), str(httpError)))
            print ("HTTP or Connection Error: " + str(httpError) + '\n Aguardar_proxima_tentativa_de_conexao \n ==========\n')
            pass
        except (Exception, json.JSONDecodeError) as jsonError:
            json_error_time = time.strftime('%Y%m%d-%H%M%S')       
            with open(logerrorpath+'datario-json-error-msg.txt','+a') as f:
                f.write("%a\n%a\n%a\n==========\n" % (json_error_time, str(response), str(jsonError)))
            print ("JSON Error: " + str(jsonError) + '\n Aguardar_proximo_bloco_de_dados \n ==========\n')
            pass
        finally:
            window_wait()